```Test mardown```

In [ ]:
import matplotlib.pyplot as plt

def plot_scatter_matrix(wine_data, good_threshold, bad_threshold, save_plot=False):
    
    
    
fig = plt.figure()  # an empty figure with no axes
fig.suptitle('No axes on this figure')  # Add a title so we know which it is

# fig, ax_lst = plt.subplots(2, 2)  # a figure with a 2x2 grid of Axes


In [13]:

plt.plot(x, x, label='linear')
plt.plot(x, x**2, label='quadratic')
plt.plot(x, x**3, label='cubic')

plt.xlabel('x label')
plt.ylabel('y label')

plt.title("Simple Plot")

plt.legend()

plt.show()

NameError: name 'x' is not defined

In [6]:
import os

for root, dirs, files in os.walk("./"):
    for filename in files:
        print(filename)

sommelier2.ipynb
scatter_matrix.png
winequality-red.csv
sommelier2-checkpoint.ipynb


In [ ]:
import itertools
import matplotlib.pyplot as plt
import csv
import copy

def main():
    fig = 0
    rownum = 0
    data = []
    with open('winequality-red.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')

        for row in csv_reader:
            data.append (row)
            rownum += 1
    
    scatterplot_matrix(data, 3, 8, True)


def scatterplot_matrix(data, good_threshold, bad_threshold, save_plot=False):

    names = data[0]
    del names[-1]

    numvars = len(names)
    fig, axes = plt.subplots(nrows=numvars, ncols=numvars, figsize=(numvars * 1.5,numvars * 1.5))
    fig.subplots_adjust(hspace=0, wspace=0)

    for ax in axes.flat:
        # Hide all ticks and labels
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        # Set up ticks only on one side for the "edge" subplots...
#         if ax.is_first_col():
#             ax.yaxis.set_ticks_position('left')
#         if ax.is_last_col():
#             ax.yaxis.set_ticks_position('right')
#         if ax.is_first_row():
#             ax.xaxis.set_ticks_position('top')
#         if ax.is_last_row():
#             ax.xaxis.set_ticks_position('bottom')

    for i, label in enumerate(names):
        j = copy.deepcopy(i)
        j -= 1
        axes[i,i].annotate(label, (0.5, 0.5), xycoords='data',
            ha='center', va='center')
        print(i)
        while j >= 0:
            for d in data[1:]:
                color = '';
                if (float(d[-1]) > good_threshold):
                    color = 'green'
                elif (float(d[-1]) < bad_threshold):
                    color = 'red'
                else:
                    continue
                axes[j, i].plot(float(d[i]), float(d[j]), linestyle='none', marker='.', markersize=2, color=color, mfc='none')
                axes[i, j].plot(float(d[j]), float(d[i]), linestyle='none', marker='.', markersize=2, color=color, mfc='none')
            j -= 1

    # Label the diagonal subplots...
#     for i, label in enumerate(names):
#         axes[i,i].annotate(label, (0.5, 0.5), xycoords='data',
#                 ha='center', va='center')
    
    if (save_plot):
        plt.savefig("scatter_matrix.png")
    
    plt.show()

main()

0
1
2
3
4
5
6
7
8
